<a href="https://colab.research.google.com/github/rubozero/mcd-2023-programming/blob/proyecto/marginalizacion_mex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>